# Analyse des Staus in NRW

Seit Anfang 2018 wird stündlich die Staulänge auf NRWs Autobahnen abgefragt.
Die Ergebnisse sind im folgenden zu sehen


In [168]:
import pandas as pd
import numpy as np

# for plotting
import altair as alt
from altair import datum

### read data

In [167]:
stau_path= "staudaten.csv"
stau = pd.read_csv(stau_path, sep=';')
stau.head()

,timestamp,deutschland,nrw
0,2018-02-11 20:40:21,"51,8",4
1,2018-02-11 21:00:12,"48,8",5
2,2018-02-11 21:10:40,"46,8",4
3,2018-02-11 21:13:12,"46,8",4
4,2018-02-11 22:00:14,"47,8",1


### clean data

In [126]:
#clean up
stau=stau[stau['timestamp']!='###################raspi############################'].copy()
stau['timestamp'] = pd.to_datetime(stau['timestamp'])
stau['nrw'] = stau['nrw'].replace('-','0')
stau=stau[['timestamp','nrw']].copy()
stau.head()

,timestamp,nrw
0,2018-02-11 20:40:21,4
1,2018-02-11 21:00:12,5
2,2018-02-11 21:10:40,4
3,2018-02-11 21:13:12,4
4,2018-02-11 22:00:14,1


In [127]:
stau['timestamp_rd']=stau['timestamp'].dt.round('1h')
stau=stau.drop_duplicates('timestamp_rd', keep='first')
stau['date']=stau['timestamp'].apply(lambda x: x.date())
stau.head()

,timestamp,nrw,timestamp_rd,date
0,2018-02-11 20:40:21,4,2018-02-11 21:00:00,2018-02-11
4,2018-02-11 22:00:14,1,2018-02-11 22:00:00,2018-02-11
5,2018-02-11 23:00:15,1,2018-02-11 23:00:00,2018-02-11
6,2018-02-12 00:00:12,2,2018-02-12 00:00:00,2018-02-12
7,2018-02-12 01:00:12,2,2018-02-12 01:00:00,2018-02-12


### prepare months borders


In [132]:
#fill missing days 

start=2018
years=5

all_days = pd.date_range('01/01/'+str(start), periods=365*years, freq='D')
all_days_df=all_days.to_frame()
all_days_df=all_days_df.reset_index()

all_days_df=all_days_df.rename(columns={0:'date'})
all_days_df= all_days_df.drop(columns=['index'])

all_days_df['weekday']=all_days_df['date'].dt.weekday
all_days_df['weekday_name']=all_days_df['date'].dt.weekday_name
all_days_df['Week_Number'] = all_days_df['date'].dt.week
all_days_df['year'] = all_days_df['date'].dt.year
all_days_df['day'] = all_days_df['date'].dt.day

all_days_df.head()

,date,weekday,weekday_name,Week_Number,year,day
0,2018-01-01,0,Monday,1,2018,1
1,2018-01-02,1,Tuesday,1,2018,2
2,2018-01-03,2,Wednesday,1,2018,3
3,2018-01-04,3,Thursday,1,2018,4
4,2018-01-05,4,Friday,1,2018,5


### prepare data availability

In [133]:
stau_cnt=stau.groupby(['date'])['nrw'].count().reset_index()
stau_cnt['date']= pd.to_datetime(stau_cnt['date'])

stau_cnt['weekday']=stau_cnt['date'].dt.weekday
stau_cnt['weekday_name']=stau_cnt['date'].dt.weekday_name
stau_cnt['Week_Number'] = stau_cnt['date'].dt.week
stau_cnt['year'] = stau_cnt['date'].dt.year
stau_cnt['day']=stau_cnt.date.dt.day

stau_cnt=stau_cnt.sort_values('weekday')

# lege letzte Dezembertage ans Ende
for index, row in stau_cnt.iterrows():
    
    if  ((row.Week_Number==1)&(row.date.month==12)):
        stau_cnt.at[index,'Week_Number']=53


### plot data availability

In [135]:

df_=stau_cnt

years= sorted(df_.year.unique())

for year in years:
    df=df_[df_.year==year]
    
    #### CHART
    chart=alt.Chart(df).mark_rect(width=18,height=18).encode(
        alt.Y('weekday:O', title='weekday', axis=None),
        alt.X('Week_Number:O', title='Kalenderwoche'),

        alt.Color('nrw',  scale=alt.Scale(scheme='spectral'), title='Datenverfügbarkeit pro Tag'), #, sort="descending"

        tooltip=[
            alt.Tooltip('date', title='Date'),
            alt.Tooltip('nrw', title='Anzahl Datenpunkte'),
            alt.Tooltip('weekday_name', title='Wochentag')
        ],
        #row=alt.Row('year', title='Jahr'),
    ).properties(
        title="Stau in NRW - Datenverfügbarkeit " + str(year)
    )
    
    
    #### MONATSGRENZEN
    df=all_days_df[all_days_df.year==year].copy()
    
    #df=df.copy()
    df['weekday_1']=df['weekday']-1
    vline=alt.Chart(df).mark_rule(color='black', strokeWidth=2, xOffset=-11, yOffset=+10, y2Offset=+9).encode(
        x='Week_Number:N',
        y=alt.Y('weekday:N', axis=None),
        y2=alt.Y2('weekday_1:N')
    ).transform_filter((datum.day <=7))


    #df=df.copy()
    df['Week_Number_1']=df['Week_Number']+1
    hline = alt.Chart(df).mark_rule(color='black', strokeWidth=2, yOffset=-11, xOffset=-12, x2Offset=-12).encode(
        x=alt.X('Week_Number:N',scale=alt.Scale(domain=list(range(1,54)))),
        x2=alt.X2('Week_Number_1:N'),
        y=alt.Y('weekday:N', axis=None,scale=alt.Scale(domain=[-1,0,1,2,3,4,5,6])),
    ).transform_filter((datum.day ==1))

    
    (chart+hline+vline).display()
    print('\n\n\n')


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

### prepare mean Staulänge


In [ ]:
stau.nrw=stau.nrw.astype(int)
stau_mean=stau.groupby(['date'])['nrw'].mean().reset_index()
stau_mean['date']= pd.to_datetime(stau_mean['date'])

stau_mean['weekday']=stau_mean['date'].dt.weekday
stau_mean['weekday_name']=stau_mean['date'].dt.weekday_name
stau_mean['Week_Number'] = stau_mean['date'].dt.week
stau_mean['year'] = stau_mean['date'].dt.year
stau_mean['day']=stau_mean.date.dt.day


stau_mean=stau_mean.sort_values('weekday')

stau_mean['nrw']= stau_mean['nrw'].round(1)

# lege letzte Dezembertage ans Ende
for index, row in stau_mean.iterrows():
    
    if  ((row.Week_Number==1)&(row.date.month==12)):
        stau_mean.at[index,'Week_Number']=53


### plot mean Staulänge


In [121]:

df_=stau_mean

years= sorted(df_.year.unique())

for year in years:
    df=df_[df_.year==year]
    #print (year)
    #### CHART
    chart=alt.Chart(df).mark_rect(width=18,height=18).encode(
        alt.Y('weekday:O', title='weekday', axis=None),
        alt.X('Week_Number:O', title='Kalenderwoche'),

        #alt.Color('nrw',  scale=alt.Scale(scheme='spectral'), title='Datenverfügbarkeit pro Tag'), #, sort="descending"
        #alt.Color('nrw:Q',  scale=alt.Scale(scheme='lightmulti'), title='Staulänge in NRW [km]'), #, sort="descending"
        color=alt.Color('nrw:Q',#scale=alt.Scale(scheme='spectral')),
                    #sort='descending'   ,
                        title='Staulänge in NRW [km]',
                        scale=alt.Scale(domain=[0, int(df_.nrw.max())],
                                        scheme='lightmulti')),

        tooltip=[
            alt.Tooltip('date', title='Date'),
            alt.Tooltip('nrw', title='durchschn. Staulänge [km]'),
            alt.Tooltip('weekday_name', title='Wochentag')
        ],
        #row=alt.Row('year', title='Jahr'),
    ).properties(
        title="Stau in NRW - durchschn. stündl. Länge " + str(year)
    )

    
    
    #### MONATSGRENZEN
    df=all_days_df[all_days_df.year==year].copy()
    
    #df=df.copy()
    df['weekday_1']=df['weekday']-1
    vline=alt.Chart(df).mark_rule(color='black', strokeWidth=2, xOffset=-11, yOffset=+10, y2Offset=+9).encode(
        x='Week_Number:N',
        y=alt.Y('weekday:N', axis=None),
        y2=alt.Y2('weekday_1:N')
    ).transform_filter((datum.day <=7))


    #df=df.copy()
    df['Week_Number_1']=df['Week_Number']+1
    hline = alt.Chart(df).mark_rule(color='black', strokeWidth=2, yOffset=-11, xOffset=-12, x2Offset=-12).encode(
        x=alt.X('Week_Number:N',scale=alt.Scale(domain=list(range(1,54)))),
        x2=alt.X2('Week_Number_1:N'),
        y=alt.Y('weekday:N', axis=None,scale=alt.Scale(domain=[-1,0,1,2,3,4,5,6])),
    ).transform_filter((datum.day ==1))

    
    (chart+hline+vline).display()
    print('\n\n\n')


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

### prepare max. Staulänge

In [118]:
stau.nrw=stau.nrw.astype(int)
stau_max=stau.groupby(['date'])['nrw'].max().reset_index()
stau_max['date']= pd.to_datetime(stau_max['date'])

stau_max['weekday']=stau_max['date'].dt.weekday
stau_max['weekday_name']=stau_max['date'].dt.weekday_name
stau_max['Week_Number'] = stau_max['date'].dt.week
stau_max['year'] = stau_max['date'].dt.year
stau_max['day']=stau_max.date.dt.day

stau_max=stau_max.sort_values('weekday')

stau_max['nrw']= stau_max['nrw'].round(1)



# lege letzte Dezembertage ans Ende
for index, row in stau_max.iterrows():
    if  ((row.Week_Number==1)&(row.date.month==12)):
        stau_max.at[index,'Week_Number']=53

### plot max Staulänge


In [119]:

df_=stau_max

years= sorted(df_.year.unique())

for year in years:
    df=df_[df_.year==year].copy()
    #print (year)
    #### CHART
    chart=alt.Chart(df).mark_rect(width=18,height=18).encode(
        y=alt.Y('weekday:O', title='weekday', axis=None),
        x=alt.X('Week_Number:O', title='Kalenderwoche'),

        #alt.Color('nrw',  scale=alt.Scale(scheme='spectral'), title='Datenverfügbarkeit pro Tag'), #, sort="descending"
        #alt.Color('nrw:Q',  scale=alt.Scale(scheme='lightmulti'), title='Staulänge in NRW [km]'), #, sort="descending"
        
        color=alt.Color('nrw:Q',#scale=alt.Scale(scheme='spectral')),
                    #sort='descending'   ,
                        title='Staulänge in NRW [km]',
                        scale=alt.Scale(domain=[0, int(df_.nrw.max())],
                                        scheme='lightmulti')),

        tooltip=[
            alt.Tooltip('date', title='Date'),
            alt.Tooltip('nrw', title='durchschn. Staulänge [km]'),
            alt.Tooltip('weekday_name', title='Wochentag')
        ],
        #row=alt.Row('year', title='Jahr'),
    ).properties(
        title="Stau in NRW - max. stündl. Länge " + str(year)
    )

    
    
    #### MONATSGRENZEN
    df=all_days_df[all_days_df.year==year].copy()
    
    #df=df.copy()
    df['weekday_1']=df['weekday']-1
    vline=alt.Chart(df).mark_rule(color='black', strokeWidth=2, xOffset=-11, yOffset=+10, y2Offset=+9).encode(
        x='Week_Number:N',
        y=alt.Y('weekday:N', axis=None),
        y2=alt.Y2('weekday_1:N')
    ).transform_filter((datum.day <=7))


    #df=df.copy()
    df['Week_Number_1']=df['Week_Number']+1
    hline = alt.Chart(df).mark_rule(color='black', strokeWidth=2, yOffset=-11, xOffset=-12, x2Offset=-12).encode(
        x=alt.X('Week_Number:N',scale=alt.Scale(domain=list(range(1,54)))),
        x2=alt.X2('Week_Number_1:N'),
        y=alt.Y('weekday:N', axis=None,scale=alt.Scale(domain=[-1,0,1,2,3,4,5,6])),
    ).transform_filter((datum.day ==1))

    
    (chart+hline+vline).display()
    print('\n\n\n')


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

## Tagesgang

In [144]:
stau_lite=stau[['timestamp','nrw']].copy()
stau_lite.head()

,timestamp,nrw
0,2018-02-11 20:40:21,4
4,2018-02-11 22:00:14,1
5,2018-02-11 23:00:15,1
6,2018-02-12 00:00:12,2
7,2018-02-12 01:00:12,2


In [148]:
stau_lite.nrw=stau_lite.nrw.astype(int)

In [149]:
stau_lite['hour']=stau_lite['timestamp'].dt.hour
stau_lite['weekday_int']=stau_lite['timestamp'].dt.weekday
stau_lite['year']=stau_lite['timestamp'].dt.year

In [150]:
stau_lite['weekday']=stau_lite['timestamp'].dt.weekday_name

In [151]:
df_group=stau_lite.groupby(['year','weekday_int','weekday','hour'])['nrw'].mean().reset_index()


In [152]:
df_group['ix']=df_group['weekday_int'].astype(str)+'_'+df_group['hour'].astype(str)

In [153]:

source=df_group#[df_group.year==2019]

chart= alt.Chart(source).mark_line().encode(
    x='hour',
    y='nrw:Q',
    #color='weekday',
    color=alt.Color('weekday',  sort=["mon", "tue", "wed", "thu", "fri", "sat", "sun"]),

    #detail='year'
    row='year'
)

chart.save('tagesgang.html')
chart

alt.Chart(...)

In [155]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [166]:
source=stau_lite[['nrw','weekday','hour','year','timestamp']]


selection = alt.selection_multi(fields=['weekday'])#, init={'weekday': 'Monday'})
opacity = alt.condition(selection, alt.value(0.4), alt.value(0.0))

color=alt.condition(selection,
                    alt.Color('weekday:N',  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], legend=None),
                    alt.value('lightgray'))


## main chart
chart= alt.Chart(source).mark_point().encode(
    x='hour',
    y='nrw:Q',
    #color='weekday',
    opacity=opacity,
    color=color,#alt.Color('weekday',  sort=["Monday", "tue", "wed", "thu", "fri", "sat", "sun"], legend=None),
    #legend=None,
   # opacity=opacity
    #opacity=alt.opacity('weekday',  sort=["Monday", "tue", "wed", "thu", "fri", "sat", "sun"]),
    #detail='year'
    tooltip=['timestamp','weekday:O','hour'],
    row='year'
).interactive()

## interactive legend
legend = alt.Chart(source).mark_rect().encode(
        y=alt.Y('weekday',  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
        color=color,#alt.Color('weekday',  sort=["Monday", "tue", "wed", "thu", "fri", "sat", "sun"], legend=None),
        tooltip=['mean(nrw):Q'],#,'mean(cnt):Q']
    ).add_selection(
    selection
).properties(
        width=20,
    )

#chart.save('tagesgang_points.html')
chart | legend

alt.HConcatChart(...)